<a href="https://colab.research.google.com/github/lab-china/nlg_eval/blob/master/00_guidance_USE2WV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#USE2WV

##driver

In [1]:
from google.colab import files
from google.colab import drive
drive.mount('/content/mount')
# ai.research.jp@gmail.com
#Ctrl + M + L

Mounted at /content/mount


##環境構築

In [2]:
%cd /content/
!pip install sudachipy  > /dev/null # 本体のインストール
!pip install sudachidict_core  > /dev/null #辞書のインストール
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
!pip install unidic-lite > /dev/null
!pip install tensorflow_text > /dev/null
! export MECABRC="/etc/mecabrc"

/content
Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [3]:
import MeCab  as mc
import subprocess
import json
import pandas as pd
import numpy as np
import readline
from gensim.models import word2vec
import MeCab
import re
import warnings
from sklearn import preprocessing
import gensim
import tensorflow_hub as hub
import tensorflow_text
cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,shell=True).communicate()[0]).decode('utf-8')
# meparse=mc.Tagger("-d {0}".format(path))
meparse = MeCab.Tagger("-Owakati -d {0}".format(path)) 
warnings.simplefilter('ignore')
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

## USE vectors make

In [ ]:
import gensim
model_w2v300 = gensim.models.KeyedVectors.load_word2vec_format('/content/mount/My Drive/00_work/fastText_w2v/model.vec', binary=False)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
words=model_w2v300.wv.vocab
wordslst=[]
for i, w in enumerate(words):
  wordslst.append(w)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [ ]:
import numpy as np
cnt_all=len(wordslst)
d=3000
print(cnt_all)
cnt=int(cnt_all/d)
if cnt_all%d>0:
  cnt=cnt+1
fn='/content/use_vec.txt'
with open(fn, "w", encoding="utf-8") as w:
    for i in range(cnt):
       print(i,cnt)
       wlst=wordslst[i*d:(i+1)*d]
       wlst_exp=np.expand_dims(wlst,axis=1)
       vector=embed(wlst)
       vector=np.round(vector,6)
       str_o=np.concatenate([wlst_exp,vector],axis=1)
       for stri in str_o:
        str_dt=' '.join(stri) + '\n'
        w.write(str_dt)
w.close()

In [ ]:
import pandas as pd
import numpy as np

from gensim.models import KeyedVectors

fn='/content/use_vec.txt'
fnout='/content/use_vec.model'
# 単語ラベルをインデックスにしてDataFrameで読み込む
# vectors = pd.read_csv(fn, delimiter=' ', index_col=0, header=None)

with open(fn, 'r') as original, open(fnout, 'w') as transformed:
    vocab_count = 335476 #len(words)  # 単語数
    size = 512  # 次元数
    transformed.write(f'{vocab_count} {size}\n')
    transformed.write(original.read())

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format(fnout, binary=False)
model.save_word2vec_format(fnout+'.bin', binary=True)

In [ ]:
##https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c
pm='/content/mount/My Drive/00_work/w2v_ja/ja.bin'
%cd /content/mount/My Drive/00_work/w2v_ja/
import gensim
# model = gensim.models.KeyedVectors.load_word2vec_format('/content/mount/My Drive/00_work/w2v_ja/ja.bin', binary=True)
model = gensim.models.KeyedVectors.load_word2vec_format('ja.bin', binary=True)
# model = gensim.models.KeyedVectors.load('/content/mount/My Drive/00_work/fastText_w2v/model.vec')
# model_w2v300 = gensim.models.KeyedVectors.load_word2vec_format('/content/mount/My Drive/00_work/fastText_w2v/model.vec', binary=False)


In [ ]:
model = gensim.models.Word2Vec.load('ja.bin')

In [ ]:
!wget https://drive.google.com/open?id=0B0ZXk88koS2KMzRjbnE4ZHJmcWM

##白ヤギコーポレーションのモデル OK

In [ ]:
%cd /content/

In [ ]:
# %cd /content/mount/My Drive/00_work
!wget http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip
!unzip latest-ja-word2vec-gensim-model.zip

In [ ]:
from gensim.models.word2vec import Word2Vec
# %cd /content/mount/My Drive/00_work
model_path = 'word2vec.gensim.model'
latest_model = Word2Vec.load(model_path)

In [ ]:
len(latest_model.wv.vocab)

335476

In [ ]:
model.most_similar('高学年')

##東北大学 乾・岡崎研究室のモデル OK

In [ ]:
%cd /content
!wget http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2
!tar -xvf 20170201.tar.bz2

In [ ]:
import gensim
fn="/content/entity_vector/entity_vector.model.bin"
jawiki_vectors=gensim.models.KeyedVectors.load_word2vec_format(fn, binary=True)

In [ ]:
jawiki_vectors.wv.most_similar('日本')

In [ ]:
#1015474
len(jawiki_vectors.wv.vocab)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


1015474

##国立国語研究所 jawiki_vectors OK

In [ ]:
# 国立国語研究所 サイズ大きすぎで、使えない
# !wget https://sudachi.s3-ap-northeast-1.amazonaws.com/chive/chive-1.2-mc5_gensim.tar.gz
# !tar zxvf  chive-1.2-mc5_gensim.tar.gz
# import gensim
# chive_vectors = gensim.models.KeyedVectors.load("/content/chive-1.2-mc5_gensim/chive-1.2-mc5.kv")
# !rm chive-1.2-mc5_gensim.tar.gz

In [ ]:
import subprocess
import json
import pandas as pd
import tensorflow_hub as hub
import numpy as np
import readline
from gensim.models import word2vec
import re
import warnings
from sklearn import preprocessing
import gensim

In [ ]:
# !wget https://sudachi.s3-ap-northeast-1.amazonaws.com/chive/chive-1.2-mc15_gensim.tar.gz
# !tar zxvf  chive-1.2-mc15_gensim.tar.gz
import gensim
chive_vectors = gensim.models.KeyedVectors.load("/content/mount/My Drive/00_work/chive-1.2-mc15_gensim/chive-1.2-mc15.kv")
chive_vectors.wv.most_similar('父親')

##singletongue/WikiEntVec NG

In [ ]:
#jawiki.all_vectors.300d.txt.bz2
%cd /content
!wget https://github.com/singletongue/WikiEntVec/releases/download/20190520/jawiki.all_vectors.300d.txt.bz2
!bzip2 -d  jawiki.all_vectors.300d.txt.bz2

In [ ]:
!git clone https://github.com/marekrei/convertvec.git
%cd convertvec 
!make

## save text to binary file

In [ ]:
# ! /content/convertvec/convertvec txt2bin 'jawiki.all_vectors.300d.txt' 'jawiki.all_vectors.300d.bin'
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/content/glove_vectors_50d.bin', binary=False)
model.save_word2vec_format('gui.glove.vectors.50d.bin', binary=True)

In [ ]:
!cp '/content/mount/My Drive/00_work/gui_glove/glove_vectors_50d.bin' /content/glove_vectors_50d.bin

In [ ]:
jawiki_vectors =  gensim.models.KeyedVectors.load_word2vec_format("/content/mount/My Drive/00_work/jawiki.all_vectors.300d.bin", binary=True)

In [ ]:
jawiki_vectors.wv.most_similar('父親')

In [ ]:
meparse.parse('小学高学年になってから、父親は、私がお風呂に入っているのを覗きにきて、性器を見せるようにいうようになりました')

##評価

In [ ]:
%cd /content/mount/My Drive/00_work
!git clone https://github.com/tmu-nlp/JapaneseWordSimilarityDataset.git
!mkdir JWSAN
%cd JWSAN
!wget http://www.utm.inf.uec.ac.jp/JWSAN/jwsan-2145.csv
!wget http://www.utm.inf.uec.ac.jp/JWSAN/jwsan-1400.csv

In [ ]:
%cd /content/mount/My Drive/00_work/
!git clone https://github.com/shihono/evaluate_japanese_w2v.git

In [ ]:
%cd /content
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.vec.gz
!gunzip cc.ja.300.vec.gz

In [ ]:
embedding_dict = gensim.models.KeyedVectors.load_word2vec_format("/content/mount/My Drive/00_work/gui_glove/glove_vectors_50d.model", binary=False) 
embedding_dict.save_word2vec_format('/content/mount/My Drive/00_work/gui_glove/gui.glove_vectors_50d.model'+".bin", binary=True)
# model = gensim.models.KeyedVectors.load_word2vec_format('/content/cc.ja.300.vec'+".bin", binary=True)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# import gensim
import sys
# reload(sys)
# sys.setdefaultencoding("ISO-8859-1")
from  datetime import datetime as dt
filelst=[
         ['/content/mount/My\ Drive/00_work/JapaneseWordSimilarityDataset/score_adj.csv',0,1,13],
         ['/content/mount/My\ Drive/00_work/JapaneseWordSimilarityDataset/score_adv.csv',0,1,13],
         ['/content/mount/My\ Drive/00_work/JapaneseWordSimilarityDataset/score_noun.csv',0,1,13],
         ['/content/mount/My\ Drive/00_work/JapaneseWordSimilarityDataset/score_verb.csv',0,1,13],
         ['/content/mount/My\ Drive/00_work/JWSAN/jwsan-1400.csv',1,2,4],
         ['/content/mount/My\ Drive/00_work/JWSAN/jwsan-2145.csv',1,2,4]
]
modLst=[
      # '/content/mount/My\ Drive/00_work/latest-ja-word2vec-gensim-model/word2vec.gensim.model' ,
      # '/content/mount/My\ Drive/00_work/entity_vector/entity_vector.model.bin' ,
      # '/content/mount/My\ Drive/00_work/chive-1.2-mc15_gensim/chive-1.2-mc15.kv' ,
      # '/content/mount/My\ Drive/00_work/chive-1.2-mc5_gensim/chive-1.2-mc5.kv' ,
      # '/content/mount/My\ Drive/00_work/word2vec/word2vec.gensim.model' ,
      # '/content/mount/My\ Drive/00_work/w2vmodel/ja-gensim.50d.data.model' ,
      #   '/content/mount/My\ Drive/00_work/jawiki.all_vectors.300d.bin' ,
      # '/content/mount/My\ Drive/00_work/fastText_w2v/model.vec' ,
      # '/content/mount/My\ Drive/00_work/facebook_fastText/cc.ja.300.vec.bin' ,
      '/content/mount/My\ Drive/00_work/gui_glove/gui.glove_vectors_50d.model.bin',

]
%cd /content/mount/My Drive/00_work/evaluate_japanese_w2v
for mod in modLst:
  print("***********",dt.utcnow().strftime( '%y/%m/%d %H:%M:%S'),{mod})
  for fn,w1,w2,sim in filelst:
    !python eval.py {mod} \
        {fn} \
        -v --col {w1} {w2} {sim} -m --mecab_dict {path}
        # -v --col {w1} {w2} {sim} -m --sudachi
  print("***********",dt.utcnow().strftime( '%y/%m/%d %H:%M:%S'),{mod})
#/content/mount/My Drive/00_work/jawiki.all_vectors.300d.bin

In [ ]:
import gensim
gensim.models.KeyedVectors.load_word2vec_format('/content/mount/My Drive/00_work/gui_glove/gui.glove_vectors_50d.model.bin', binary=True,encoding='latin1', unicode_errors='ignore')
# gensim.models.KeyedVectors.load('/content/mount/My Drive/00_work/gui_glove/glove_vectors_50d.txt')
# embedding_dict = gensim.models.KeyedVectors.load_word2vec_format('/content/cc.ja.300.bin', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
dtpath='/content/mount/My Drive/00_work/gui_glove'
# 単語ラベルをインデックスにしてDataFrameで読み込む
vectors = pd.read_csv(dtpath+'/glove_vectors_50d.txt', delimiter=' ', index_col=0, header=None)

with open(dtpath+'/glove_vectors_50d.txt', 'r') as original, open(dtpath+'/glove_vectors_50d.model', 'w') as transformed:
    vocab_count = vectors.shape[0]  # 単語数
    size = vectors.shape[1]  # 次元数

    transformed.write(f'{vocab_count} {size}\n')
    transformed.write(original.read())

#モデル訓練データを作成

In [ ]:
!wget https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-meta-current.xml.bz2

In [ ]:
!git clone https://github.com/attardi/wikiextractor.git
!pip install wikiextractor

In [ ]:
!python -m wikiextractor.WikiExtractor  '/content/jawiki-latest-pages-meta-current.xml.bz2'  -b 500K -o /content/extracted

In [ ]:
%cd /content/all_dt
!zip -r wikki_pages_meda.zip wikki_pages_meda


In [ ]:
!cp '/content/all_dt/wikki_pages_meda.zip' '/content/mount/My Drive/00_work/01_glove/02_wiki_t/wikki_pages_meda.zip'

## create wikki data

In [ ]:
txtpath='/content/extracted/'
dtpath='/content/all_dt/wikki_pages_meda/'
from glob import glob
import os
# livedoorニュースコーパスを全て分かち書きして１つのファイルに書き込む
# カテゴリを配列で取得
categories = [name for name in os.listdir(txtpath) if os.path.isdir(txtpath +name)]
cnt=len(categories)
for i, cat in enumerate(categories):
  print(cat,i,cnt)
  with open(dtpath+'/jawiki_pages_meta%06d.txt'%(i), "w", encoding="utf-8") as w:
    path = txtpath + cat + "/*"
    files = glob(path)

    for fn in files:
      with open(fn, "r", encoding="utf-8") as f:
        data = f.read()
        w.write(data + '\n')
      f.close()
    w.close()

#モデル訓練実施

## create corpus

In [ ]:
import re, MeCab
from glob import glob
# 分かち書きはMeCabで適当に
# tagger = MeCab.Tagger("-Owakati")
cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,shell=True).communicate()[0]).decode('utf-8')
tagger = MeCab.Tagger("-Owakati -d {0}".format(path)) 
def make_wakati(sentence):
    # MeCabで分かち書き
    sentence = tagger.parse(sentence)
    # 半角全角英数字除去
    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
    # 記号もろもろ除去
    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
    # スペースで区切って形態素の配列へ
    wakati = sentence.split(" ")
    # 空の要素は削除
    wakati = list(filter(("").__ne__, wakati))
    return wakati

In [ ]:
import os
txtpath='/content/all_dt/'
dtpath='/content/mount/My Drive/00_work/01_glove/INES_guidance/'
# livedoorニュースコーパスを全て分かち書きして１つのファイルに書き込む
# カテゴリを配列で取得
categories = [name for name in os.listdir(txtpath) if os.path.isdir(txtpath +name)]
cnt=len(categories)
with open(dtpath+'/gui_corpus.txt', "w", encoding="utf-8") as w:
    for i, cat in enumerate(categories):
        path = txtpath + cat + "/*"
        files = glob(path)
        for fn in files:
            print(cat,i,cnt,fn)
            with open(fn, "r", encoding="utf-8") as f:
                data = f.read()
                datalst=data.split('\n')
                for dti in  datalst:
                  if len(dti.strip()) <5:
                         continue
                  wakati = make_wakati(dti)
                  str_dt=' '.join(wakati) + '\n'
                  w.write(str_dt)
w.close()

## create vocabulary

In [ ]:
%cd /content/mount/My Drive/00_work/01_glove 
!chmod 777 ./build/vocab_count
! ./build/vocab_count \
  -min-count 1 \
  -verbose 2 < {dtpath}/wiki_corpus.txt > {dtpath}/gui_vocab.txt


In [ ]:
!chmod 777 ./build/cooccur
! ./build/cooccur \
   -memory 12 \
   -vocab-file {dtpath}/gui_vocab.txt \
   -verbose 2 -window-size 15 < {dtpath}/wiki_corpus.txt  > {dttmp}/cooccurrence.txt

In [ ]:
!chmod 777 ./build/shuffle
! ./build/shuffle \
  -memory 12 \
  -verbose 2 \
  < {dttmp}/cooccurrence.txt \
  > {dttmp}/cooccurrence_shuffle

In [ ]:
%cd  '/content/mount/My Drive/00_work/01_glove/'
!chmod 777 ./build/glove
! ./build/glove \
   -save-file {dtpath}/glove_vectors_300d \
   -threads 50 \
   -input-file {dttmp}/cooccurrence_shuffle \
   -x-max 15 \
   -iter 30 \
   -vector-size 300 \
   -binary 2 \
   -vocab-file {dtpath}/gui_vocab.txt \
   -verbose 2